In [1]:
import os
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
pwd

In [2]:
cd /content/drive/MyDrive/GomokuAI

/content/drive/MyDrive/GomokuAI


# Git

In [ ]:
!git pull

remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 6 (delta 5), reused 6 (delta 5), pack-reused 0
Unpacking objects: 100% (6/6), 1.02 KiB | 0 bytes/s, done.
From https://github.com/onetwothr1/GomokuAI
   3428df1..bd64d3e  main       -> origin/main
Updating 3428df1..bd64d3e
Fast-forward
 agent/alphazero_agent.py | 23 +++++++++++------------
 train.py                 | 20 ++++++++++++++------
 utils.py                 | 35 +++++++++++++++++++----------------
 3 files changed, 44 insertions(+), 34 deletions(-)


In [ ]:
!git add .

# Code

In [ ]:
!python train.py -m "models/alphazero 0.pt" -d "experience/alphazero 0 self-play 750x4.pickle" -n 10000 -lr 0.001 --lr-decay 0.95 --early-stop 10

alphazero 0 80.pt를 선택! -> alphazero 750
750 vs 0 => 9:1

c=1.0 vs c=2.0 => 12:2 (p-value 0.012)

c=1.0 vs c=1.5 => 10:0

In [ ]:
import importlib
import bot_v_bot, utils, agent.alphazero_agent, compare_performance, self_play
# import net.alphazero_net
# importlib.reload(bot_v_bot)
# importlib.reload(compare_performance)
importlib.reload(utils)
importlib.reload(self_play)
# importlib.reload(agent.alphazero_agent)
# importlib.reload(net.alphazero_net)

<module 'self_play' from '/content/drive/MyDrive/GomokuAI/self_play.py'>

In [ ]:
!python bot_v_bot.py

In [ ]:
!python compare_performance.py --model1 "models/alphazero 750.pt" --model2 "models/alphazero 750.pt" -n 10 --verbose 4

In [ ]:
for i in range(100):
  !python self_play.py -n 10 -m "models/alphazero 750.pt" --file-num {i} -rollout 100 -c 1 --noise-intensity 0.25 --alpha 0.1 --reward-decay 0.92 --verbose 0

10it [10:20, 62.01s/it]
Traceback (most recent call last):
  File "/content/drive/MyDrive/GomokuAI/self_play.py", line 118, in <module>
    main()
  File "/content/drive/MyDrive/GomokuAI/self_play.py", line 108, in main
    with open('experience/%s self-play %d.txt' %(save_path(args.model, args.num_games, args.file_num, True)), 'w') as file:
TypeError: not enough arguments for format string
10it [08:00, 48.09s/it]
Traceback (most recent call last):
  File "/content/drive/MyDrive/GomokuAI/self_play.py", line 118, in <module>
    main()
  File "/content/drive/MyDrive/GomokuAI/self_play.py", line 108, in main
    with open('experience/%s self-play %d.txt' %(save_path(args.model, args.num_games, args.file_num, True)), 'w') as file:
TypeError: not enough arguments for format string
10it [10:23, 62.38s/it]
Traceback (most recent call last):
  File "/content/drive/MyDrive/GomokuAI/self_play.py", line 118, in <module>
    main()
  File "/content/drive/MyDrive/GomokuAI/self_play.py", line 108, 